# Question 3: Trajectory Evaluation and g2o

_Refer to the example notebooks for installation instructions_

# Evo

So you've implemented 2D SLAM, great! Now, what? We need a measure of how good the trajectory is. The error/loss used earlier doesn't tell us much about how the trajectory differs from the ground truth. Here, we try to do just this - compute error metrics. Rather than computing these from scratch, we will just Evo - https://github.com/MichaelGrupp/evo/.

Look at the absolute pose error (APE) and relative pose error (RPE). What do they capture and how are they calculated (descriptive answer)? How do these metrics differ in methodology? Can we determine if the error is more along the x/y axis?

Answer the above questions and report errors for the obtained trajectory.

In [ ]:
# Write up with plots/images

If you're interested, play around with this tool and add any other plots that you think might be relevant/interesting.

## Analysis using Evo

Analysis of the trajectory generated in question 2 is explored in this section. The method, commands, inference, etc. are also explored


### Setup and Generation

To install [Evo](https://github.com/MichaelGrupp/evo/) the following command was run

```bash
pip install evo
```

After this, binary files get generated in the `bin` folder of the Python environment. Post this, the following commands were run

#### Generating Kitti file for Ground Truth

The following commands were run to generate the ground truth [.kitti](https://github.com/MichaelGrupp/evo/wiki/Formats#kitti---kitti-dataset-pose-format) file. The starting path is the project root (folder for `Project1`)

```bash
cd misc
cp ../data/gt.txt ./ground-truth.g2o
python ./g2o_to_kitti.py ground-truth.g2o gt.kitti
```

This generates a `gt.kitti` that contains the ground truth poses as transforms of the form `[R|t]` (12 numbers per row as described in format).


#### Generating Kitti file for calibrated data

The calibrated data generated as a result of Question 2 is converted to a `.kitti` file using the following commands (in the same `misc` folder as before)

```bash
cp ../questions/results/q2/edges-calibrated.g2o ./
python ./g2o_to_kitti.py edges-calibrated.g2o opt.kitti
```

This generates a `opt.kitti` file that contains calibrated poses as shown in the image below. This is compared in the next section.

![Calibrated Trajectory](./results/q2/p2/lm-out-1.png)

### Trajectory analysis

This is done using the following command

```bash
evo_traj kitti gt.kitti opt.kitti -v --plot --plot_mode xy
```



The resultant output is

```txt
--------------------------------------------------------------------------------
Loaded 120 poses from: gt.kitti
Loaded 120 poses from: opt.kitti
--------------------------------------------------------------------------------
name:	gt
infos:
	nr. of poses	120
	path length (m)	52.976517997999295
	pos_end (m)	[-2.8 -4.5  0. ]
	pos_start (m)	[-8.  5.  0.]
--------------------------------------------------------------------------------
name:	opt
infos:
	nr. of poses	120
	path length (m)	54.66919770066222
	pos_end (m)	[-0.63074972  1.07208598  0.        ]
	pos_start (m)	[-8.00001143  4.99998875  0.        ]
```

The errors seem big (in meters), at least the path length is more or less the same. The figures generated are shown here and explained


#### Trajectory Plot

![Trajectory plot](./results/q3/evo/traj_1.png)

This shows the ground truth and the optimal trajectory (obtained through `question 2`) in comparison,


#### XYZ Plot

The plot is shown below

![XYZ plot](./results/q3/evo/traj_2.png)

It shows the X, Y and Z values of the points in the ground truth and optimal trajectory. As expected, the optimal values aren't exactly matching, but they're closely following the ground truth (sometimes not very closely). This is the result of optimization.


#### RPY View

The plot is shown below

![RPY plot](./results/q3/evo/traj_3.png)

As expected, there is only yaw used here. Even here, the optimized trajectory is more or less following (roughly) the ground truth. However, as described in question 2, there appears to be errors in this (the ground truth), which cannot be fit easily.


### Relative Pose Error

This is done using the following command

```bash
evo_rpe kitti gt.kitti opt.kitti -v --plot --plot_mode xy
```


The resultant output is

```txt
--------------------------------------------------------------------------------
Loaded 120 poses from: gt.kitti
Loaded 120 poses from: opt.kitti
--------------------------------------------------------------------------------
Found 119 pairs with delta 1 (frames) among 120 poses using consecutive pairs.
Compared 119 relative pose pairs, delta = 1 (frames) with consecutive pairs.
Calculating RPE for translation part pose relation...
--------------------------------------------------------------------------------
RPE w.r.t. translation part (m)
for delta = 1 (frames) using consecutive pairs
(not aligned)

       max	0.292755
      mean	0.118914
    median	0.112302
       min	0.001513
      rmse	0.134819
       sse	2.162955
       std	0.063526

--------------------------------------------------------------------------------
```

The relative pose error is over a fixed time interval. It being less than a meter is slightly acceptable.

# g2o

Install g2o as mentioned in `examples/g2o.ipynb` and optimise `edges.txt`, the file you used earlier. Also use `g2o_viewer` and optimize `intel` (a trajectory in the Intel research lab) and `sphere`. They should look something like:


<table><tr>
<td> <img src="../misc/intel.jpg" alt="Drawing" style="width: 250px;"/> </td>
<td> <img src="../misc/sphere.jpg" alt="Drawing" style="width: 250px;"/> </td>
</tr></table>

Write briefly about your observations and try out few options in the GUI. What do they do, how do they perform?

In [ ]:
# Your answer